In [190]:
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torch import nn
from numpy import array
import numpy as np
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

In [111]:
def yield_tokens(data):
    for doc in data:
        yield tokenizer(doc)

In [16]:
# source text
data = """ Jack and Jill went up the hill\n
To fetch a pail of water\n
Jack fell down and broke his crown\n
And Jill came tumbling after\n """

In [199]:
tokenizer = get_tokenizer("basic_english")
encoded = [tokenizer(text) for text in [data]][0]

vocab_size = len(encoded)
sequences = list()
for i in range(1, len(encoded)):
  sequence = encoded[i-1:i+1]
  sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

vocab = build_vocab_from_iterator(yield_tokens(y))

# encoding
encoded_y = [vocab(tokenizer(doc)) for doc in y]
encoded_X = [vocab(tokenizer(doc)) for doc in X]

y_tensor = torch.tensor(encoded_y)
X_tensor = torch.tensor(encoded_X)

print(encoded_y,'\n')
print(encoded_X,'\n')

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

Total Sequences: 24
[[0], [1], [20], [18], [15], [10], [16], [9], [2], [14], [13], [19], [12], [8], [7], [0], [4], [11], [6], [0], [1], [5], [17], [3]] 

[[12], [0], [1], [20], [18], [15], [10], [16], [9], [2], [14], [13], [19], [12], [8], [7], [0], [4], [11], [6], [0], [1], [5], [17]] 



In [153]:
class LM(nn.Module):
  def __init__(self,vocab_size):
    super(LM,self).__init__()
    self.embed = nn.Embedding(vocab_size,10)
    self.lstm = nn.LSTM(10,50)
    self.lin1 = nn.Linear(50,vocab_size)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input):
    embedded = self.embed(input)
    lstm_out, _ = self.lstm(embedded)
    output = self.lin1(lstm_out[-1])
    return self.softmax(output)

In [207]:
model = LM(vocab_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 500

# train
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in dataloader:
        outputs = model(X_batch)

        # hajtépés és fájdalom
        print(f'X_batch shape: {X_batch.shape}')
        print(f'y_batch shape: {y_batch.shape}')
        print(f'outputs shape: {outputs.shape}')


        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# eval
with torch.no_grad():
    model.eval()
    X_tensor = dataset.data
    y_tensor = dataset.labels

    outputs = model(X_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_tensor).float().mean()
    print('Accuracy: %f' % (accuracy.item() * 100))

X_batch shape: torch.Size([2, 1])
y_batch shape: torch.Size([2, 1])
outputs shape: torch.Size([1, 25])


ValueError: Expected input batch_size (1) to match target batch_size (2).